In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
from collections import Counter

%matplotlib inline

/home/samuel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = pd.read_csv("../final_with_labels.csv")
data_matrix = data.as_matrix()

In [3]:
data.head()

,ID,pubcsnum,mar_stat,sex,primsite,histo3v,beho3v,grade,dx_conf,csexten,...,maligcount,benbordcount,year_dx,nhiade,codpub,srv_time_mon,srv_time_mon_flag,surv_1,surv_2,surv_5
0,1,7011365,9,2,C184,8210,3,2,1,200,...,4,0,2010,7,0,71,1,1,1,1
1,2,7014780,5,2,C209,8140,3,2,1,200,...,2,0,2007,6,21040,36,1,1,1,0
2,3,7017183,2,1,C186,8140,2,9,1,50,...,3,0,2012,6,21050,37,1,1,1,0
3,4,7017183,2,1,C209,8263,3,2,1,455,...,3,0,2012,6,21050,37,1,1,1,0
4,5,7018423,2,1,C186,8263,3,3,1,400,...,3,0,2011,2,0,52,1,1,1,1


In [4]:
data_matrix.shape

(46785, 33)

In [ ]:
data_matrix[0]

# Preprocessing

In [5]:
Y1 = data_matrix[:,-3]
Y2 = data_matrix[:,-2]
Y5 = data_matrix[:,-1]

In [6]:
X = data_matrix[:,2:24]
X_cat = X[:,:-5]
# primary site is only categorical variable with string labels
X_primsite = X[:,2]
X_cont = X[:,-5:]
# X_cat contains only categorical variables with integer labels
X_cat = np.delete(X_cat, [2], axis=1)

In [7]:
# Fill missing values in three eval columns with 9: unknown
X_cat[:,9] = np.array([9 if np.isnan(x) else x for x in X_cat[:,9]])
X_cat[:,10] = np.array([9 if np.isnan(x) else x for x in X_cat[:,10]])
X_cat[:,11] = np.array([9 if np.isnan(x) else x for x in X_cat[:,11]])

In [8]:
# one-hot encode primary site
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
X_primsite_1hot = encoder.fit_transform(X_primsite)
encoder.classes_

array(['C180', 'C181', 'C182', 'C183', 'C184', 'C185', 'C186', 'C187',
       'C188', 'C189', 'C199', 'C209', 'C260'], dtype='<U4')

In [9]:
# one-hot encode remaining categorical variables
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
X_cat_1hot = encoder.fit_transform(X_cat)

In [10]:
# combine all categorical variables
X_cat = np.hstack((X_primsite_1hot, X_cat_1hot))

The continuous variables in order: age_dx, eod10_pn, eod10_ne, cstumsiz, maligcount. Age and maligcount do not reuqire imputation, and eod10_ne (nodes examined) will not be used as a variable.


Here we will impute values for eod10_pn and cstumsiz. We will implement multivariate imputation by chained equations (MICE) on these two continuous variables. Their indices are and values which need to be imputed are:

1: eod10_pn (95, 97, 98, 99)<br>
3: cstumsiz (998, 999)<br>

Additionally, we will replace the values (991, 992, 993, 994, 995) in cstumsiz with (1, 2, 3, 4, 5) respectively.


### Note: The above should be re-verified for each different dataset. This is for colorectal cancer in Hispanic patients after 2004.

In [11]:
# implements MICE imputation with linear regression model on eod10_pn and cstumsiz
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression

NUM_CYCLES=10

missing_values_pn = (95, 97, 98, 99)
missing_values_tumsiz = (998, 999)

pn = np.array(X_cont[:,1]).reshape(-1,1)
tumsiz = np.array(X_cont[:,3]).reshape(-1,1)

# non-MICE imputation
for i, size in enumerate(tumsiz):
    if size not in (990, 991, 992, 993, 994, 995):
        continue
    elif size == 990:
        tumsiz[i] = 0
    elif size == 991:
        tumsiz[i] = 1
    elif size == 992:
        tumsiz[i] = 2
    elif size == 993:
        tumsiz[i] = 3
    elif size == 994:
        tumsiz[i] = 4
    elif size == 995:
        tumsiz[i] = 5


# set missing values to -1
for i, value in enumerate(pn):
    if value in missing_values_pn:
        pn[i] = -1
for i, value in enumerate(tumsiz):
    if value in missing_values_tumsiz:
        tumsiz[i] = -1

# indices for valid and imputation-needed positions in pn and tumsiz
idx_pn_missing = np.where(pn == -1)[0]
idx_pn_valid = np.where(pn != -1)[0]
idx_tumsiz_missing = np.where(tumsiz == -1)[0]
idx_tumsiz_valid = np.where(tumsiz != -1)[0]
    
# set missing values to mean
# copy = false modifies array in place
imp_pn = Imputer(missing_values = -1, copy=False)
imp_pn.fit_transform(pn)
imp_tumsiz = Imputer(missing_values = -1, copy=False)
imp_tumsiz.fit_transform(tumsiz)


for i in np.arange(NUM_CYCLES):
    print("Cycle " + str(i+1))
    
    # fit pn
    X_ind = np.hstack((X_cat[idx_pn_valid], tumsiz[idx_pn_valid]))
    X_rep = np.hstack((X_cat[idx_pn_missing], tumsiz[idx_pn_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, pn[idx_pn_valid])
    pn[idx_pn_missing] = lin_reg.predict(X_rep)
    pn = np.maximum(pn, 0)
    pn = np.minimum(pn, 100)
    
    # fit tumsiz
    X_ind = np.hstack((X_cat[idx_tumsiz_valid], pn[idx_tumsiz_valid]))
    X_rep = np.hstack((X_cat[idx_tumsiz_missing], pn[idx_tumsiz_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, tumsiz[idx_tumsiz_valid])
    tumsiz[idx_tumsiz_missing] = lin_reg.predict(X_rep)
    tumsiz = np.maximum(tumsiz, 0)
    tumsiz = np.minimum(tumsiz, 990)


Cycle 1
Cycle 2
Cycle 3
Cycle 4
Cycle 5
Cycle 6
Cycle 7
Cycle 8
Cycle 9
Cycle 10


In [ ]:
pn = pn.reshape(-1,).astype(int)
Counter(pn)

In [ ]:
print(np.mean(pn[idx_pn_valid]))
print(np.std(pn[idx_pn_valid]))
print(np.mean(pn[idx_pn_missing]))
print(np.std(pn[idx_pn_missing]))

In [ ]:
print(np.mean(tumsiz[idx_tumsiz_valid]))
print(np.std(tumsiz[idx_tumsiz_valid]))
print(np.mean(tumsiz[idx_tumsiz_missing]))
print(np.std(tumsiz[idx_tumsiz_missing]))

In [ ]:
from collections import Counter
Counter(X_cont[:,4])

# Training

For hispanic colrect, there doesn't seem to be any categorical values in malig.

In [12]:
age_dx = np.array(X_cont[:,0])
malig = np.array(X_cont[:,4])
X_all = np.column_stack((X_cat, age_dx, pn, malig, tumsiz))
print(X_all.shape)

(46785, 313)


In [14]:
# shuffle
np.random.seed(97)
idx = np.random.permutation(len(X_all))
X = X_all[idx]
Y = Y5[idx]

TEST_SET_SIZE = 6000

X_train, X_test = X[:-TEST_SET_SIZE], X[-TEST_SET_SIZE:]
Y_train, Y_test = Y[:-TEST_SET_SIZE].astype(int), Y[-TEST_SET_SIZE:].astype(int)

# Feature Scaling

In [15]:
# feature scaling: scale features based on training data only
from sklearn.preprocessing import StandardScaler, MinMaxScaler
std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(X_train)
#mm_scaler = MinMaxScaler()
#X_train = mm_scaler.fit_transform(X_train)

/home/samuel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
# min_sample_split: 300,400
# min_samples_leaf: 200
# max_depth: 130
# min_weight_fraction_leaf: .01
param_grid = [{'min_samples_leaf':[300,400,500]}]
tree_clf_reg = DecisionTreeClassifier()
grid_search = GridSearchCV(tree_clf_reg, param_grid, cv=5, scoring="roc_auc", verbose=1)
grid_search.fit(X_train, Y_train.astype(int))

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(grid_search.best_params_))

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
Y_pred_test = grid_search.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

## K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, Y_train.astype(int))

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
Y_pred_test = knn_clf.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

## Logistic Regression

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = [{'penalty':['l2'], 'C':[0.1]}]
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring="roc_auc", n_jobs=-1, verbose=5)
grid_search.fit(X_train, Y_train.astype(int))

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(grid_search.best_params_))

In [ ]:
grid_search.best_score_

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
Y_pred_test = grid_search.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

## Bagging

In [27]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
        DecisionTreeClassifier(), n_estimators=500,
        max_samples=1000, bootstrap=True, n_jobs=-1,
)

bag_clf.fit(X_train, Y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1000, n_estimators=500, n_jobs=-1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [ ]:
# probably add cross-validation?

In [28]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
Y_pred_test = bag_clf.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

/home/samuel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


0.8246666666666667
ROC: 0.7200112367929936


array([[ 746,  691],
       [ 361, 4202]])

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(X_train, Y_train)

In [ ]:
y_pred_rf = rnd_clf.predict(X_test)
accuracy_score(y_pred_rf, Y_test)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import SVC

svm_clf = Pipeline((
                        ("scaler", StandardScaler()),
                        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
))
svm_clf.fit(X_train, Y_train)

In [ ]:
accuracy_score(svm_clf.predict(X_test), Y_test)
accuracy_score(svm_clf.predict(X_train), Y_train)